In [1]:
!pip install gpytorch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [2]:
!pip install bert_score
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 34.5 MB/s eta 0:00:00:00:0100:01


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
import pandas as pd

from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)

from gpytorch.models import ExactGP
from gpytorch.means import ZeroMean
from gpytorch.kernels import Kernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.distributions import MultivariateNormal
from gpytorch.mlls import ExactMarginalLogLikelihood

from bert_score import score as bert_score
from typing import List, Optional
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

2025-05-01 17:12:21.302643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746119541.478180      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746119541.529798      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# ================== NEW DIFFUSION COMPONENTS ==================
class DiffusionRefiner:
    def __init__(self, model_name: str = "google/flan-t5-base"):
        """Text refinement using diffusion-inspired process"""
        self.refiner = pipeline(
            "text2text-generation", 
            model=model_name, 
            device=0 if torch.cuda.is_available() else -1
        )
        self.paraphraser = pipeline(
            "text2text-generation",
            model="humarin/chatgpt_paraphraser_on_T5_base",
            device=0 if torch.cuda.is_available() else -1
        )

    def refine(self, texts: List[str], steps: int = 2) -> List[str]:
        """Apply diffusion-inspired refinement to a batch of texts."""
        noised = self.paraphraser(
            texts,
            do_sample=True,
            top_k=50,
            num_return_sequences=1,
            max_length=512
        )
        noised_texts = [item["generated_text"] for item in noised]
    
        refined = self.refiner(
            noised_texts,
            max_length=512,
            num_beams=3,
            early_stopping=True
        )
        refined_texts = [item["generated_text"] for item in refined]
    
        # Optionally apply probabilistic skip of refinement
        final_output = [
            refined if np.random.rand() < 0.3 else original
            for original, refined in zip(texts, refined_texts)
        ]
        return final_output

In [7]:
# example of text diffusion

test_diff = DiffusionRefiner()
text_ex = "an apple a day keeps the doctor away".split()
print(test_diff.refine(text_ex))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


['an', 'apple', 'a', 'day', 'People have to fight for their freedom.', 'the', 'If he gets a cold or a flu, they can treat him.', 'away']


In [8]:
# ================== MODIFIED PERTURBATION GENERATOR ==================
class PerturbationGenerator:
    def __init__(self, model_name: str = "t5-base", use_diffusion: bool = True):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.diffusion = DiffusionRefiner() if use_diffusion else None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def perturb(self, text: str, num_samples: int = 9) -> List[str]:
        """Generate perturbations with optional diffusion refinement"""
        # Generate initial perturbations with T5
        inputs = self.tokenizer(
            f"paraphrase: {text}", 
            return_tensors="pt", 
            max_length=512, 
            truncation=True
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            num_return_sequences=num_samples,
            do_sample=True,
            max_length=512,
        )
        
        raw_perturbations = [text] + [
            self.tokenizer.decode(out, skip_special_tokens=True) 
            for out in outputs
        ]
        
        # Apply diffusion refinement if enabled
        if self.diffusion:
            try:
                refined = self.diffusion.refine(raw_perturbations[1:])
                return [raw_perturbations[0]] + refined
            except Exception as e:
                print(f"Diffusion refinement failed: {e}, using raw perturbations")
        
        return raw_perturbations
    

In [9]:
class BayesianDetector:
    def __init__(
        self,
        source_model,
        perturbation_model: str = "t5-base",
        num_total_samples: int = 10,
        uncertainty_threshold: float = 0.1,
        use_diffusion: bool = True
    ):
        self.num_total_samples = num_total_samples
        self.source_model = source_model
        self.perturbator = PerturbationGenerator(
            perturbation_model, 
            use_diffusion=use_diffusion
        )
        self.gp = None
        self.all_texts = []
        self.uncertainty_threshold = uncertainty_threshold
        self.sim_matrix_cache = {}

    # ================== HELPER METHODS ==================
    def _compute_similarity_matrix(self, texts: List[str]) -> torch.Tensor:
        """Precompute BERTScore similarity matrix with caching"""
        cache_key = hash(tuple(texts))
        if cache_key in self.sim_matrix_cache:
            return self.sim_matrix_cache[cache_key]
        
        n = len(texts)
        sim_matrix = torch.zeros((n, n))
        for i in tqdm(range(n), desc="Computing Similarities"):
            for j in range(i, n):
                if i == j:
                    sim_matrix[i, j] = 1.0
                else:
                    _, _, F1 = bert_score(
                        [texts[i]], [texts[j]],
                        lang="en",
                        model_type="bert-base-uncased",
                        rescale_with_baseline=True,
                        verbose=False
                    )
                    sim_matrix[i, j] = sim_matrix[j, i] = F1.item()
        
        self.sim_matrix_cache[cache_key] = sim_matrix
        return sim_matrix


    # ================== CORE DETECTION LOGIC ==================
    def detect(self, text: str, max_queries: int = 5) -> float:
        """Enhanced detection with diffusion-guided perturbations"""
        self.all_texts = self.perturbator.perturb(
            text, 
            num_samples=self.num_total_samples-1
        )
        max_queries = min(max_queries, self.num_total_samples - 2)
        
        # Precompute similarity matrix
        sim_matrix = self._compute_similarity_matrix(self.all_texts)
        
        # Active sampling loop
        train_indices = torch.tensor([0, 1])
        y_train = self.source_model.get_log_prob([self.all_texts[0], self.all_texts[1]])
        
        for _ in tqdm(range(max_queries - 2), desc="Active Sampling"):
            self.gp = SurrogateModel(sim_matrix, train_indices, y_train)
            self.gp.optimize_hyperparameters(train_indices, y_train)
            
            remaining_indices = torch.tensor(
                [i for i in range(len(self.all_texts)) if i not in train_indices]
            )
            
            with torch.no_grad():
                self.gp.eval()
                pred = self.gp.likelihood(self.gp(remaining_indices))
                uncertainties = pred.variance
                
            # Early stopping condition
            if torch.max(uncertainties) < self.uncertainty_threshold:
                break
                
            next_idx = remaining_indices[torch.argmax(uncertainties)].item()
            train_indices = torch.cat([train_indices, torch.tensor([next_idx])])
            y_train = torch.cat([y_train, self.source_model.get_log_prob([self.all_texts[next_idx]])])
        
        # Final prediction
        with torch.no_grad():
            self.gp.eval()
            all_preds = self.gp(torch.arange(len(self.all_texts)))
            
        original_score = all_preds.mean[0]
        perturbed_scores = all_preds.mean[1:]
        perturbed_std = perturbed_scores.std()
        
        # Stability guard
        if perturbed_std < 1e-8:
            perturbed_std = 1e-8
            
        return (original_score - perturbed_scores.mean()) / perturbed_std
    

In [10]:
class BertScoreKernel(Kernel):
    def __init__(self, sim_matrix: torch.Tensor):
        super().__init__()
        self.sim_matrix = sim_matrix
        self.register_parameter(
            name="raw_alpha", 
            parameter = nn.Parameter(torch.tensor(0.0)))
        self.register_parameter(
            name="raw_beta", 
            parameter = nn.Parameter(torch.tensor(0.0)))
        
    @property
    def alpha(self):
        return F.softplus(self.raw_alpha)

    @property
    def beta(self):
        return F.softplus(self.raw_beta)

    def forward(self, x1: torch.Tensor, x2: torch.Tensor, **kwargs) -> torch.Tensor:
        x1 = x1.long().squeeze()
        x2 = x2.long().squeeze()
        
        x1_idx = x1.unsqueeze(-1)
        x2_idx = x2.unsqueeze(0)
        
        K = self.alpha * self.sim_matrix[x1_idx, x2_idx] + self.beta
        return K

In [11]:
class SurrogateModel(ExactGP):
    def __init__(self, sim_matrix: torch.Tensor, train_indices: torch.Tensor, train_y: torch.Tensor):
        likelihood = GaussianLikelihood()
        super().__init__(train_indices, train_y, likelihood)
        self.mean_module = ZeroMean()    #  Can also try ConstantMean()
        self.covar_module = BertScoreKernel(sim_matrix)
        
    def forward(self, x: torch.Tensor) -> MultivariateNormal:
        x = x.long()
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

    def optimize_hyperparameters(self, train_x, train_y, lr=0.01, iterations=50):   #   May tune this "iteration" to 100
        self.train()
        self.likelihood.train()
        
        optimizer = Adam([
            {'params': self.covar_module.parameters()},
            {'params': self.likelihood.parameters()}
        ], lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=iterations)
        
        mll = ExactMarginalLogLikelihood(self.likelihood, self)

        for _ in range(iterations):
            optimizer.zero_grad()
            output = self(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            scheduler.step()

In [12]:
class SourceModel:
    def __init__(self, model_name="gpt2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()
        
    def get_log_prob(self, texts: List[str]) -> torch.Tensor:
        log_probs = []
        with torch.no_grad():
            for text in texts:
                inputs = self.tokenizer(text, return_tensors="pt", truncation=True).to(self.device)
                
                # Handle empty input
                if inputs["input_ids"].shape[1] == 0:
                    log_probs.append(0.0)
                    continue
                
                # Handle single-token edge case
                num_tokens = inputs["input_ids"].shape[1]
                if num_tokens == 1:
                    log_probs.append(0.0)
                    continue
                
                outputs = self.model(**inputs, labels=inputs["input_ids"])
                logits = outputs.logits
                labels = inputs["input_ids"]
                
                log_probs_seq = torch.log_softmax(logits, dim=-1)[:, :-1, :]
                log_probs_text = torch.gather(
                    log_probs_seq, 
                    dim=2, 
                    index=labels[:, 1:, None]
                ).squeeze(-1).sum(dim=1)

                normalized_log_prob = log_probs_text.item() / (num_tokens - 1)  
                log_probs.append(normalized_log_prob)
            
        print(log_probs)    
        return torch.tensor(log_probs)

In [13]:
source_model = SourceModel(model_name="gpt2")  

print("-------Diffusion Refinement-------")
# With diffusion refinement
detector_diff = BayesianDetector(
    source_model,
    perturbation_model="t5-base",
    use_diffusion=True,
    num_total_samples=10,
    uncertainty_threshold=1e-3
)

print("-------Without Diffusion Refinement-------")
# Without diffusion refinement
detector_raw = BayesianDetector(
    source_model,
    use_diffusion=False,
    num_total_samples=10,
    uncertainty_threshold=1e-3
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


-------Without Diffusion Refinement-------


In [14]:
# Example Text
test_text = "The rapid development of artificial intelligence has brought both opportunities and challenges to modern society."

print("Running detection with diffusion refinement...")
score_diff = detector_diff.detect(test_text)
print(f"Diffusion-enhanced score: {score_diff:.2f}")

print("\nRunning detection without diffusion refinement...")
score_raw = detector_raw.detect(test_text)
print(f"Raw T5 score: {score_raw:.2f}")

Running detection with diffusion refinement...


Computing Similarities:   0%|          | 0/10 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

[-3.7087020874023438]


Active Sampling:  67%|██████▋   | 2/3 [00:00<00:00,  7.94it/s]

[-3.4789760589599608]


Active Sampling: 100%|██████████| 3/3 [00:00<00:00,  7.95it/s]

[-3.550840277420847]
Raw T5 score: 0.37


In [15]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")
        print(f"Total memory: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2:.2f} MB")
        print(f"Allocated memory: {torch.cuda.memory_allocated(0) / 1024 ** 2:.2f} MB")
        print(f"Reserved memory: {torch.cuda.memory_reserved(0) / 1024 ** 2:.2f} MB")
        print(f"Free (unreserved) memory: {(torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)) / 1024 ** 2:.2f} MB")
    else:
        print("CUDA not available.")

print_gpu_memory()

GPU Name: Tesla P100-PCIE-16GB
Total memory: 16269.25 MB
Allocated memory: 5789.70 MB
Reserved memory: 6724.00 MB
Free (unreserved) memory: 934.30 MB


In [16]:
dataset = pd.read_csv('/kaggle/input/llm-human-classification-data/LLM-Human Classification Data.csv')
dataset.shape

(29145, 2)

In [17]:
llm_dataset = dataset[dataset['generated'] == 1]
human_dataset = dataset[dataset['generated'] == 0]
print(llm_dataset.shape, human_dataset.shape)

llm_dataset = llm_dataset[:int((llm_dataset.shape[0])*0.05)]
human_dataset = human_dataset[:int((human_dataset.shape[0])*0.03)]
print(llm_dataset.shape, human_dataset.shape)

smaller_dataset = pd.concat([llm_dataset, human_dataset])
smaller_dataset = smaller_dataset.sample(frac=1)
smaller_dataset.shape

(11637, 2) (17508, 2)
(581, 2) (525, 2)


(1106, 2)

In [18]:
print(smaller_dataset['generated'].value_counts())
smaller_dataset.head()

generated
1    581
0    525
Name: count, dtype: int64


,text,generated
144,"In recent years, there has been a growing tren...",1
1254,Many people in today's world depend on cars to...,0
148,Limiting car usage can have several advantages...,1
1270,The Electoral College has been around since th...,0
344,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1


In [19]:
torch.cuda.empty_cache()
import sys
import os
import contextlib

scores_diff = []
scores_raw = []

sentences = list(smaller_dataset['text'])

# Open a null writer for stdout only
null_output = open(os.devnull, 'w')

for num, sentence in enumerate(sentences, 1):
    if num <= 5:
        scores_diff.append(detector_diff.detect(sentence).item())
        scores_raw.append(detector_raw.detect(sentence).item())
        print(f"Detection Scores for sentence {num} is: {scores_diff[-1]:0.4f} and {scores_raw[-1]:0.4f}")
    else:
        # Suppress only stdout; stderr (errors/exceptions) will still show
        with contextlib.redirect_stdout(null_output):
            try:
                scores_diff.append(detector_diff.detect(sentence).item())
                scores_raw.append(detector_raw.detect(sentence).item())
                
            except Exception as e:
                print(f"Error on sentence {num}: {e}", file=sys.stderr)
                
        print(f"Detection Scores for sentence {num} is: {scores_diff[-1]:0.4f} and {scores_raw[-1]:0.4f}")

    torch.cuda.empty_cache()

null_output.close()


Computing Similarities: 100%|██████████| 10/10 [00:21<00:00,  2.16s/it]


[-2.562668750330478, 0.0]


Active Sampling:  33%|███▎      | 1/3 [00:00<00:00,  7.87it/s]

[-2.73518443107605]


Active Sampling:  67%|██████▋   | 2/3 [00:00<00:00,  7.87it/s]

...

Detection Scores for sentence 263 is: 1.9681 and 3.5429


Active Sampling: 100%|██████████| 3/3 [00:00<00:00,  8.04it/s]


Detection Scores for sentence 264 is: 1.4655 and 1.0541


Computing Similarities:  30%|███       | 3/10 [00:13<00:31,  4.55s/it]


KeyboardInterrupt: 

In [20]:
print_gpu_memory()

GPU Name: Tesla P100-PCIE-16GB
Total memory: 16269.25 MB
Allocated memory: 5789.70 MB
Reserved memory: 6836.00 MB
Free (unreserved) memory: 1046.30 MB


In [28]:
score_dataset = smaller_dataset.copy()
def pad_scores(scores, target_len):
    return scores + [None] * (target_len - len(scores))

# Pad both score lists to match the length of the dataset
target_len = len(score_dataset)
scores_diff_padded = pad_scores(scores_diff, target_len)
scores_raw_padded = pad_scores(scores_raw, target_len)
score_dataset["score_with_diffusion"] = scores_diff_padded
score_dataset["score_without_diffusion"] = scores_raw_padded
score_dataset.to_csv("/kaggle/working/score_diffusion_dataset.csv", index=False)

In [29]:
score_dataset.head()

,text,generated,score_with_diffusion,score_without_diffusion
144,"In recent years, there has been a growing tren...",1,-0.537769,-6.113869e+01
1254,Many people in today's world depend on cars to...,0,1.075490,9.730217e-01
148,Limiting car usage can have several advantages...,1,1.538736,2.690150e+00
1270,The Electoral College has been around since th...,0,-0.169041,-8.213293e+01
344,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1,-1.643560,-1.193706e+08
